## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-07-30 15:26:14.497553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value (200) and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_value_counts[application_value_counts <= 200].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts[classification_value_counts > 1]


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value (150) and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts <= 150].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categories = application_df.select_dtypes(include=['object']).columns.tolist()
application_df_encoded = pd.get_dummies(application_df, columns=categories)


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  6
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 282       
                                                                 
 dense_1 (Dense)             (None, 5)                 35        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 323 (1.26 KB)
Trainable params: 323 (1.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 873us/step - loss: 0.6296 - accuracy: 0.6632
Epoch 2/100
804/804 [==============================] - 1s 823us/step - loss: 0.5775 - accuracy: 0.7226
Epoch 3/100
804/804 [==============================] - 1s 836us/step - loss: 0.5651 - accuracy: 0.7276
Epoch 4/100
804/804 [==============================] - 1s 836us/step - loss: 0.5616 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 1s 856us/step - loss: 0.5592 - accuracy: 0.7294
Epoch 6/100
804/804 [==============================] - 1s 828us/step - loss: 0.5584 - accuracy: 0.7297
Epoch 7/100
804/804 [==============================] - 1s 886us/step - loss: 0.5575 - accuracy: 0.7298
Epoch 8/100
804/804 [==============================] - 1s 845us/step - loss: 0.5564 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 1s 843us/step - loss: 0.5555 - accuracy: 0.7300
Epoch 10/100
804/804 [==============================] - 1s 834us/step - l

804/804 [==============================] - 1s 991us/step - loss: 0.5465 - accuracy: 0.7347
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7335
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7344
Epoch 83/100
804/804 [==============================] - 1s 915us/step - loss: 0.5463 - accuracy: 0.7336
Epoch 84/100
804/804 [==============================] - 1s 950us/step - loss: 0.5464 - accuracy: 0.7350
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7338
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7342
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7348
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5463 - accuracy: 0.7349
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accu

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7269 - 250ms/epoch - 932us/step
Loss: 0.5555994510650635, Accuracy: 0.7268804907798767


In [16]:
# Export our model to HDF5 file
from keras.models import save_model

# Save the model to an HDF5 file
nn.save("nn.keras")